In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D, merge, Dropout
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a 10 legtöbb képpel rendelkező alkotót kiválogatjuk
authors_to_select = author_stat[-100:]
authors_to_select

[('342d7068656b00878b23e1031a3ddf50', 165),
 ('358474690052f63f9107c823b7a61686', 165),
 ('db85b84c1c01fa143b9294f00f7acbeb', 167),
 ('30134467c49a3350f74b4311506f8386', 171),
 ('795985c5c0e7c704a354a487bdffeef5', 171),
 ('a087f33967e659c63924aa07c89b948f', 172),
 ('24ed88087b0ffeb763e89a197a5cde0d', 172),
 ('3a320926ef941d53716127548523450c', 172),
 ('ed9cf03b71a8a71d41ecb4e8406a0877', 172),
 ('a7b6670e2d23f415998849a8fdedae18', 174),
 ('8b1800319323ab9b0ce2e0b11b3f4772', 175),
 ('3b4ee6d6140803ebef405983f23ffcc1', 176),
 ('56221362cba30b7e0c96111cc38c5fcd', 176),
 ('79f6b47483600c140c91e901f746a368', 176),
 ('83518d02afc5754dd9e79985f56c3355', 178),
 ('9c948f31fc16aca442cc91e7fe05fb95', 179),
 ('b31c8ec49c09506f17ec3c0c04fce567', 179),
 ('062244b785e58518f2f189329633c59e', 181),
 ('63e73f693e541f53d98ed6c93d03ef40', 185),
 ('39c465a1d078121873a31807ce9dc2ed', 185),
 ('ea2beb11e25ec317263983dbdd6de81e', 186),
 ('1d6d112c140cb6d1d763a7ca73f2eec0', 187),
 ('74a1dad53214d04514269a188f166

In [3]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, "/media/bence/121A62041A61E4E7/learn/train")

2000 images loaded
4000 images loaded
6000 images loaded
8000 images loaded
10000 images loaded
12000 images loaded
14000 images loaded
16000 images loaded
18000 images loaded
20000 images loaded
22000 images loaded
24000 images loaded
26000 images loaded
27348 images in the result


In [4]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[23 42 71 ..., 58 23 95]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [5]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [6]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót


In [7]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
style1 = base_model.layers[54].output
style1 = GlobalAveragePooling2D()(style1)
style1 = Dense(96, activation='relu')(style1)
style2 = base_model.layers[117].output
style2 = GlobalAveragePooling2D()(style2)
style2 = Dense(160, activation='relu')(style2)
style3 = base_model.layers[184].output
style3 = GlobalAveragePooling2D()(style3)
style3 = Dense(320, activation='relu')(style3)

# egymás mellé tesszük a különböző szintű feature-öket
ff = merge([style1, style2, style3], mode='concat')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(2048, activation='relu')(ff)
ff = Dropout(0.5)(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf
tf
tf


In [8]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=50, validation_split=0.2, callbacks=[history])

Train on 21878 samples, validate on 5470 samples
Epoch 1/50
21878/21878 [==============================] - 576s - loss: 3.7353 - acc: 0.1198 - val_loss: 3.6202 - val_acc: 0.2245
Epoch 2/50
21878/21878 [==============================] - 572s - loss: 3.1318 - acc: 0.2322 - val_loss: 3.3232 - val_acc: 0.2934
Epoch 3/50
21878/21878 [==============================] - 574s - loss: 2.8861 - acc: 0.2836 - val_loss: 3.1690 - val_acc: 0.3302
Epoch 4/50
21878/21878 [==============================] - 574s - loss: 2.7382 - acc: 0.3189 - val_loss: 3.1057 - val_acc: 0.3581
Epoch 5/50
21878/21878 [==============================] - 574s - loss: 2.6271 - acc: 0.3420 - val_loss: 3.0163 - val_acc: 0.3817
Epoch 6/50
21878/21878 [==============================] - 574s - loss: 2.5438 - acc: 0.3573 - val_loss: 2.9128 - val_acc: 0.3832
Epoch 7/50
21878/21878 [==============================] - 574s - loss: 2.4903 - acc: 0.3736 - val_loss: 2.8350 - val_acc: 0.3978
Epoch 8/50
21878/21878 [========================

KeyboardInterrupt: 

In [ ]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [10]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [ ]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=100, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 21878 samples, validate on 5470 samples
Epoch 1/100
 8208/21878 [==========>...................] - ETA: 258s - loss: 1.7744 - acc: 0.5367

In [ ]:
predicted_values = model.predict(np.asarray(train_images)).shape

In [ ]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

In [ ]:
encoder.inverse_transform(festok)

In [ ]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]